# Лабороторная работа 1- Regression

# Regression 

Данные о жилье в Калифорнии

Цель работы - аппроксимировать среднюю стоимость жилья (medianHouseValue) в каждом районе по оставшимся признакам (housingMedianAge, totalRooms, totalBedrooms, population, households, medianIncome).
 

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import sklearn

Признаки:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

In [82]:
data = pd.read_csv('cal_housing_clean.csv')
print(data.head(5))
print(data.shape)

   housingMedianAge  totalRooms  totalBedrooms  population  households  \
0              41.0       880.0          129.0       322.0       126.0   
1              21.0      7099.0         1106.0      2401.0      1138.0   
2              52.0      1467.0          190.0       496.0       177.0   
3              52.0      1274.0          235.0       558.0       219.0   
4              52.0      1627.0          280.0       565.0       259.0   

   medianIncome  medianHouseValue  
0        8.3252          452600.0  
1        8.3014          358500.0  
2        7.2574          352100.0  
3        5.6431          341300.0  
4        3.8462          342200.0  
(20640, 7)


### Масштабирование признаков

In [143]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data = scaler.transform(data)
data

array([[0.78431373, 0.02233074, 0.01986344, ..., 0.02055583, 0.53966842,
        0.90226638],
       [0.39215686, 0.18050257, 0.17147734, ..., 0.18697583, 0.53802706,
        0.70824656],
       [1.        , 0.03726029, 0.02932961, ..., 0.02894261, 0.46602805,
        0.69505074],
       ...,
       [0.31372549, 0.05727657, 0.07510863, ..., 0.07104095, 0.08276438,
        0.15938285],
       [0.33333333, 0.04725571, 0.06331471, ..., 0.05722743, 0.09429525,
        0.14371281],
       [0.29411765, 0.07078183, 0.09543762, ..., 0.08699227, 0.13025338,
        0.15340349]])

In [173]:
data2 = pd.DataFrame(data=data, columns=['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'] )
print(data2.head())

   housingMedianAge  totalRooms  totalBedrooms  population  households  \
0          0.784314    0.022331       0.019863    0.008941    0.020556   
1          0.392157    0.180503       0.171477    0.067210    0.186976   
2          1.000000    0.037260       0.029330    0.013818    0.028943   
3          1.000000    0.032352       0.036313    0.015555    0.035849   
4          1.000000    0.041330       0.043296    0.015752    0.042427   

   medianIncome  medianHouseValue  
0      0.539668          0.902266  
1      0.538027          0.708247  
2      0.466028          0.695051  
3      0.354699          0.672783  
4      0.230776          0.674638  


Модель линейной регрессии - *Y = Xa* где *Y-* вектор размера 20640, *X-* Матрица размера 20640x7 и  *a-* Вектор размера 7 

In [172]:
X = data[:,0:6] 
Y = data[:,6]

In [169]:
print(Y)

[0.90226638 0.70824656 0.69505074 ... 0.15938285 0.14371281 0.15340349]


In [170]:
tf_features = tf.placeholder(tf.float32, shape=[None,6])
tf_targets = tf.placeholder(tf.float32,shape=[None,])

In [171]:
a= tf.Variable(tf.random_normal([6,1], dtype=np.float32))
Y_pred=tf.matmul(tf_features, a) 

error = tf.reduce_sum(tf.square(tf_targets-Y_pred))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train = optimizer.minimize(error)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
rand_ind = np.random.randint(len(X), size=6)

for e in range (100):        
    feed = {tf_features:X[rand_ind], tf_targets:Y[rand_ind]}
    sess.run(train, feed_dict = feed)
print("parameter a:")
print(sess.run(a))

parameter a:
[[-3.2716394e+16]
 [-2.6168224e+15]
 [-3.2348174e+15]
 [-1.2887173e+15]
 [-2.9178161e+15]
 [-1.3396990e+16]]


# С Помочью tf.estimator.LinearRegressor

In [183]:
data2 = pd.DataFrame(data=data, columns=['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'], )
print(data2.head())

linear_features = ['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population','households', 'medianIncome']
numeric_features = [tf.feature_column.numeric_column(key = column) for column in linear_features]

y_data = data2['medianHouseValue']
X_data = data2[linear_features]


   housingMedianAge  totalRooms  totalBedrooms  population  households  \
0          0.784314    0.022331       0.019863    0.008941    0.020556   
1          0.392157    0.180503       0.171477    0.067210    0.186976   
2          1.000000    0.037260       0.029330    0.013818    0.028943   
3          1.000000    0.032352       0.036313    0.015555    0.035849   
4          1.000000    0.041330       0.043296    0.015752    0.042427   

   medianIncome  medianHouseValue  
0      0.539668          0.902266  
1      0.538027          0.708247  
2      0.466028          0.695051  
3      0.354699          0.672783  
4      0.230776          0.674638  


In [184]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     X_data, y_data, test_size=0.33, random_state=42)

In [187]:
training_input_fn = tf.estimator.inputs.pandas_input_fn(x = X_train,
                                                        y=y_train,
                                                        batch_size=32,
                                                        shuffle= True,
                                                        num_epochs = None)

linear_regressor = tf.estimator.LinearRegressor(feature_columns=numeric_features,
                                                model_dir = "linear_regressor")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'linear_regressor', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F75E8B9BA8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [214]:
linear_regressor.train(input_fn = training_input_fn,steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from linear_regressor\model.ckpt-200
INFO:tensorflow:Saving checkpoints for 201 into linear_regressor\model.ckpt.
INFO:tensorflow:loss = 0.73002285, step = 201
INFO:tensorflow:Saving checkpoints for 300 into linear_regressor\model.ckpt.
INFO:tensorflow:Loss for final step: 1.7951431.


In [216]:
eval_input_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,
                                                    y=y_test,
                                                    batch_size=32,
                                                    shuffle=False,
                                                    num_epochs = 1)
pred = list(linear_regressor.predict(input_fn = eval_input_fn))
pred = [p['predictions'][0] for p in pred]
print(pred)

INFO:tensorflow:Restoring parameters from linear_regressor\model.ckpt-300
[0.20535369, 0.328702, 0.55521995, 0.5379819, 0.43827033, 0.44999152, 0.551335, 0.43474007, 0.4251662, 0.86213845, 0.29704815, 0.38366622, 0.29971012, 0.3347494, 0.5525786, 0.3932572, 0.4658116, 0.41875905, 0.45179394, 0.2600864, 0.5979524, 0.5706127, 0.4682442, 0.65814865, 0.41532767, 0.2759235, 0.32831237, 0.3916889, 0.38722765, 0.6594003, 0.2577748, 0.2579633, 0.2975061, 0.6949685, 0.44932848, 0.5726718, 0.47821864, 0.39651963, 0.4245335, 0.3576807, 0.62649184, 0.38781616, 0.25108957, 0.39027494, 0.37861666, 0.364161, 0.4126021, 0.3806065, 0.3875647, 0.40549308, 0.43215364, 0.3387098, 0.38392445, 0.20942286, 0.36952746, 0.49134102, 0.25210822, 0.55625504, 0.48031217, 0.54901886, 0.4855151, 0.38729241, 0.2514779, 0.3384512, 0.6257952, 0.38677886, 0.35232466, 0.37257197, 0.28409836, 0.34369516, 0.3757843, 0.54740036, 0.33062816, 0.4165706, 0.39171347, 0.35860366, 0.66125005, 0.40211725, 0.48641825, 0.3673569, 0.

In [221]:
from sklearn.metrics import mean_squared_error
print("RMSE:",mean_squared_error(y_test, pred))

RMSE: 0.027421286937877307
